In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

In [ ]:
from wall_analysis import parse_experiments
exp_name = '2020_12_18_flying'; 
fname = f'results/{exp_name}_real.pkl'

try:
    df_total = pd.read_pickle(fname)
    print('read', fname)
except:
    answer = input('Run wall_analysis.py to parse experiments? (y/[n])') or 'n'
    if answer == 'y':
        df_total = parse_experiments(exp_name)
        pd.to_pickle(df_total, fname)
        print('saved', fname)

In [ ]:
df_total.tail()

# Compute distance estimates

TODO: this part is hacky and will be improved by getting imu estimates rather than flow dx/dy estimates, which turn out to be very unreliable.

In [ ]:
MIN_S = 5
MAX_S = 25
VELOCITY = 0.05
D_START = 0.6 # m

df_total = df_total.assign(d_estimate=None) # initialize df_estimate
for i, row in df_total.iterrows():
    fig, ax = plt.subplots()
    ax.plot(row.positions[:, 0], label='x', color='C0')
    ax.plot(row.positions[:, 1], label='y', color='C1')
    ax.plot(row.positions[:, 2], label='z', color='C2')
    #ax.plot(row.dx)
    #ax.plot(row.dy)
    ax.set_ylim(-50, 50)
    #ax.set_xlim(0, 1000)
    ax.set_xlabel('position idx [-]')
    ax.set_ylabel('x, y [mm]')
    ax.legend(loc='lower left')
    ax.set_title("lateral movement, " + row.source + row.appendix)
    
    plt.figure()
    #plt.plot(row.yaw_deg)
    plt.plot(row.seconds, row.z, color='C2')
    plt.xlabel('time [s]')
    plt.ylabel('z [mm]')
    #plt.ylim(-90, 90)
    plt.ylim(0, 600)
    #plt.xlim(MIN_S, MAX_S)
    
    row.z[np.isnan(row.z)] = 0
    start_idx = np.where(row.seconds > MIN_S)[0][0]
    end_idx = np.where(row.seconds < MAX_S)[0][-1]
    
    assert end_idx > start_idx, f"{start_idx}<={end_idx}"
    valid_z = row.z[start_idx:end_idx]
    max_idx = start_idx + np.argmax(valid_z)
    min_idx = start_idx + np.where(valid_z > 300)[0][-1]
    duration = row.seconds[min_idx] - row.seconds[max_idx]
    plt.axvline(row.seconds[start_idx], color='r')
    plt.axvline(row.seconds[max_idx], color='k')
    plt.axvline(row.seconds[min_idx], color='k')
    plt.axvline(row.seconds[end_idx], color='b')
    plt.xlim(0, 35)
    plt.title(f"duration: {duration:.1f}s")
    
    d = np.full(len(row.seconds), np.nan)
    times = row.seconds[max_idx:min_idx] - row.seconds[max_idx]
    d[max_idx:min_idx] = D_START - times * VELOCITY
    plt.figure()
    plt.scatter(row.seconds, d)
    plt.ylim(0, 0.6)
    
    df_total.loc[i, 'd_estimate'] = d

In [ ]:
from wall_analysis import filter_by_dicts
filter_dict = {
    'source': 'mono3875',
    'appendix': '_new'
}
rows =filter_by_dicts(df_total, [filter_dict])
assert len(rows) == 1
row = rows.iloc[0]